In [1]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import pandas as pd
import json
import h5py
import os

import keras
from keras.models import load_model
from keras.datasets import cifar10
from keras.layers import Input, Dense, Conv2D, MaxPooling2D, UpSampling2D, \
                            BatchNormalization, Activation, Dropout
from keras.models import Model
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.optimizers import Adam
import pickle

from keras.layers import Dense, Input
from keras.layers import Conv2D, Flatten, Lambda
from keras.layers import Reshape, Conv2DTranspose
from keras.datasets import mnist
from keras.losses import mse, binary_crossentropy
from keras.utils import plot_model
from keras import backend as K

import numpy as np
import matplotlib.pyplot as plt
import argparse
import seaborn as sns


from sklearn.manifold import TSNE

Using TensorFlow backend.


In [4]:
for i in os.listdir('album_images')[:20]:
    print('album_images'+'/'+i)
#    cv2.imread('album_images'+'/'+i)

album_images/Awake Is The New Sleep_Ben Lee.png
album_images/The Offspring_The Offspring.png
album_images/Nothing Was The Same (Deluxe)_Drake.png
album_images/For You_Selena Gomez.png
album_images/With Ears to See and Eyes to Hear_Sleeping with Sirens.png
album_images/The Big Picture_Big L.png
album_images/Unfamiliar Faces_Matt Costa.png
album_images/Termination Bliss_Deathstars.png
album_images/Spirit_Leona Lewis.png
album_images/Sparkle And Fade_Everclear.png
album_images/When Broken Is Easily Fixed_Silverstein.png
album_images/Black Stone Cherry_Black Stone Cherry.png
album_images/Cure for Pain_Morphine.png
album_images/Who's Greatest Hits_The Who.png
album_images/To Lose My Life_White Lies.png
album_images/The Man Who_Travis.png
album_images/Innervisions_Stevie Wonder.png
album_images/Kellie Pickler_Kellie Pickler.png
album_images/King Of The Delta Blues Singers_Robert Johnson.png
album_images/Title of Record_Filter.png


In [ ]:
def get_autoencoder():
    input_img = Input(shape=(32, 32, 3))
    x = Conv2D(64, (3, 3), padding='same')(input_img)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(32, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = MaxPooling2D((2, 2), padding='same')(x)
    x = Conv2D(16, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    encoded = MaxPooling2D((2, 2), padding='same', name="encoded")(x)

    x = Conv2D(16, (3, 3), padding='same')(encoded)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(32, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(64, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = UpSampling2D((2, 2))(x)
    x = Conv2D(3, (3, 3), padding='same')(x)
    x = BatchNormalization()(x)
    decoded = Activation('sigmoid')(x)
    
    model = Model(input_img, decoded)
    model.compile(optimizer='adam', loss='binary_crossentropy')
    return model

In [ ]:
batch_size = 32
num_classes = 10
epochs = 10
x_train = truncated_images

autoencoder = get_autoencoder()
history = autoencoder.fit(x_train, x_train,
                    batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    shuffle=True)